In [ ]:
# 동적 사이저로 백테스트 실행
from datetime import datetime

import backtrader as bt

from src.backtest.backtest_builder import BacktestBuilder
from src.backtest.commission_config import CommissionConfig
from src.backtest.data_feed import PandasDataFeedConfig
from src.backtest.data_feed.candle_loader import candles_to_dataframe, timeframe_to_korean
from src.backtest.sizer_config import SizerConfig
from src.backtest.strategy.ema_simple_alignment_strategy import EmaSimpleAlignmentStrategy
from src.backtest.strategy.split_strategy import SplitStrategy
from src.container import ApplicationContainer

container = ApplicationContainer()

ticker_name = "KRW-BTC"

ticker_repository = container.ticker_repository()
ticker = ticker_repository.find_by_ticker(ticker_name)

repo = container.candle_daily_repository()
# repo = container.candle_hour1_repository()
candles = repo.get_candles(
    ticker_id=ticker.id,  # KRW-BTC
    start_datetime=datetime(2000, 1, 1),
    # start_datetime=datetime(2025, 1, 1),
    # end_datetime=datetime(2020, 10, 1)
)

# 2. DataFrame 변환 (타임프레임 자동 감지)
df, timeframe = candles_to_dataframe(candles)
config = PandasDataFeedConfig.create(df, name=ticker.ticker)

initial_cash = 100_000_000

result = (
    BacktestBuilder()
    .with_initial_cash(initial_cash)
    .with_slippage(0.0001)
    .with_commission(CommissionConfig.stock(0.0005))
    # .with_cheat_on_open()  # 시가 체결 (현재 bar의 시가로 체결)

    # ============================ 전략 ============================
    # == Buy & Hold ==
    # .with_strategy(BuyAndHoldStrategy)

    # == 단순 정배열 ==
    .with_strategy(
        EmaSimpleAlignmentStrategy,
        ema_periods=(5, 20, 40),
        # enable_short=False,
        # enable_gap_filter=True,
        # min_gap=1.0,
    )

    # == 변동성 돌파 ==
    # .with_strategy(VolatilityBreakoutStrategy, k_value=0.5)

    # == 오전/오후 ==
    # .with_strategy(TimedHoldStrategy, entry_hour=0, exit_hour=12)

    # == 20,40 정배열 & 간격 & 기울기 ==
    # .with_strategy(
    #     EmaAlignmentStrategy,
    #     min_gap=2.0,  # 최소 간격 (%)
    #     min_slope=1.0,  # 최소 기울기 (%)
    #     slope_period=5,  # 기울기 계산 기간
    # )

    # == 스플릿 ==
    # .with_strategy(
    #     SplitStrategy,
    #       split_count=10,        # 자금을 10개로 분할
    #       take_profit_rate=0.025, # 각 분할 +3% 익절
    #       trigger_rate=0.05,     # 이전 분할 -5% 시 다음 분할 진입
    # )

    # ============================ 비중 ============================

    # 전체
    # .with_sizer(SizerConfig.all_in())
    .with_sizer(SizerConfig.percent(95))

    # 20, 40일 기울기 & 간격에 따라서 비중 조절
    # .with_sizer(SizerConfig.custom(EmaDynamicSizer,
    #                                slope_period=5,
    #                                max_slope=5.0,
    #                                max_gap=10.0,
    #                                min_weight=0.3,
    #                                max_weight=1.0
    #                                ))
    .add_data(config)
    .with_analyzer(bt.analyzers.SharpeRatio, "sharpe")
    .with_analyzer(bt.analyzers.DrawDown, "drawdown")
    .with_analyzer(bt.analyzers.TradeAnalyzer, "trades")
    .with_analyzer(bt.analyzers.Returns, "returns")
    .with_analyzer(bt.analyzers.SQN, "sqn")
    .run()
)

In [ ]:
# 리포트

print("\n" + "=" * 60)
print("백테스트 결과")
print("=" * 60)

strategy = result[0]

# 데이터 기간
start_date = df.index[0]
end_date = df.index[-1]

print(f"\n[데이터 정보]")
print(f"   - 타임프레임: {timeframe_to_korean(timeframe)}")
print(f"   - 시작일: {start_date.strftime('%Y-%m-%d')}")
print(f"   - 종료일: {end_date.strftime('%Y-%m-%d')}")

# 샤프 비율
sharpe = strategy.analyzers.getbyname('sharpe').get_analysis()
print(f"\n[샤프 비율]: {sharpe.get('sharperatio', 'N/A')}")

# 수익률
returns = strategy.analyzers.getbyname('returns').get_analysis()
final_value = strategy.broker.getvalue()
total_return = (final_value - initial_cash) / initial_cash * 100

# CAGR 계산 (연복리 수익률)
years = (end_date - start_date).days / 365.25
cagr = ((final_value / initial_cash) ** (1 / years) - 1) * 100

print(f"\n[수익률]")
print(f"   - 초기 자본: {initial_cash:,.0f}원")
print(f"   - 최종 자본: {final_value:,.0f}원")
print(f"   - 총 수익률: {total_return:.2f}%")
print(f"   - 연복리 수익률 (CAGR): {cagr:.2f}%")
print(f"   - 투자 기간: {years:.1f}년")

# 최대 낙폭
drawdown = strategy.analyzers.getbyname('drawdown').get_analysis()
mdd_bars = drawdown.get('max', {}).get('len', 0)

# 타임프레임에 따라 일수 계산
if timeframe == "1m":
    mdd_days = mdd_bars / (60 * 24)  # 분 → 일
elif timeframe == "1h":
    mdd_days = mdd_bars / 24  # 시간 → 일
else:  # 1d
    mdd_days = mdd_bars

print(f"\n[최대 낙폭 (MDD)]")
print(f"   - 최대 낙폭: {drawdown.get('max', {}).get('drawdown', 0):.2f}%")
print(f"   - 최대 낙폭 기간: {mdd_days:.1f}일 ({mdd_bars:,} bars)")

# 거래 분석
trades = strategy.analyzers.getbyname('trades').get_analysis()
total_trades = trades.get('total', {}).get('total', 0)
won = trades.get('won', {}).get('total', 0)
lost = trades.get('lost', {}).get('total', 0)

print(f"\n[거래 분석 - 전체]")
print(f"   - 총 거래 수: {total_trades}")
print(f"   - 승리: {won}회")
print(f"   - 패배: {lost}회")
print(f"   - 승률: {won / total_trades * 100:.1f}%" if total_trades > 0 else "   - 승률: N/A")

# 평균 수익/손실 및 손익비
avg_won = 0
avg_lost = 0
if won > 0:
    avg_won = trades.get('won', {}).get('pnl', {}).get('average', 0)
    print(f"   - 평균 수익: {avg_won:,.0f}원")
if lost > 0:
    avg_lost = trades.get('lost', {}).get('pnl', {}).get('average', 0)
    print(f"   - 평균 손실: {avg_lost:,.0f}원")
if avg_won > 0 > avg_lost:
    profit_loss_ratio = avg_won / abs(avg_lost)
    print(f"   - 손익비: {profit_loss_ratio:.2f}")

# === 롱/숏 분리 통계 ===
long_trades = trades.get('long', {})
short_trades = trades.get('short', {})

# 롱 통계
long_total = long_trades.get('total', 0)
long_won = long_trades.get('won', 0)
long_lost = long_trades.get('lost', 0)
long_pnl = long_trades.get('pnl', {})
long_pnl_avg = long_pnl.get('average', 0)
long_won_pnl_avg = long_pnl.get('won', {}).get('average', 0)
long_lost_pnl_avg = long_pnl.get('lost', {}).get('average', 0)

print(f"\n[거래 분석 - 롱]")
print(f"   - 총 거래 수: {long_total}")
if long_total > 0:
    print(f"   - 승리: {long_won}회")
    print(f"   - 패배: {long_lost}회")
    print(f"   - 승률: {long_won / long_total * 100:.1f}%")
    if long_won > 0:
        print(f"   - 평균 수익: {long_won_pnl_avg:,.0f}원")
    if long_lost > 0:
        print(f"   - 평균 손실: {long_lost_pnl_avg:,.0f}원")
    if long_won_pnl_avg > 0 and long_lost_pnl_avg < 0:
        long_profit_loss_ratio = long_won_pnl_avg / abs(long_lost_pnl_avg)
        print(f"   - 손익비: {long_profit_loss_ratio:.2f}")

# 숏 통계
short_total = short_trades.get('total', 0)
short_won = short_trades.get('won', 0)
short_lost = short_trades.get('lost', 0)
short_pnl = short_trades.get('pnl', {})
short_pnl_avg = short_pnl.get('average', 0)
short_won_pnl_avg = short_pnl.get('won', {}).get('average', 0)
short_lost_pnl_avg = short_pnl.get('lost', {}).get('average', 0)

print(f"\n[거래 분석 - 숏]")
print(f"   - 총 거래 수: {short_total}")
if short_total > 0:
    print(f"   - 승리: {short_won}회")
    print(f"   - 패배: {short_lost}회")
    print(f"   - 승률: {short_won / short_total * 100:.1f}%")
    if short_won > 0:
        print(f"   - 평균 수익: {short_won_pnl_avg:,.0f}원")
    if short_lost > 0:
        print(f"   - 평균 손실: {short_lost_pnl_avg:,.0f}원")
    if short_won_pnl_avg > 0 and short_lost_pnl_avg < 0:
        short_profit_loss_ratio = short_won_pnl_avg / abs(short_lost_pnl_avg)
        print(f"   - 손익비: {short_profit_loss_ratio:.2f}")

# SQN (System Quality Number)
sqn = strategy.analyzers.getbyname('sqn').get_analysis()
sqn_value = sqn.get('sqn', 0)
print(f"\n[SQN (System Quality Number)]: {sqn_value:.2f}")
if sqn_value >= 2.5:
    print("   -> 우수한 전략")
elif sqn_value >= 1.6:
    print("   -> 좋은 전략")
elif sqn_value >= 0:
    print("   -> 보통 전략")
else:
    print("   -> 개선 필요")

print("\n" + "=" * 60)

In [ ]:
# 포트폴리오 가치 변화 그래프
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd

# ========== 마커 표시 옵션 ==========
SHOW_MARKERS = True  # True: 진입/청산 마커 표시, False: 마커 숨김
# ====================================

# 한글 폰트 설정 (macOS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 위에서 실행한 결과 사용
strategy = result[0]

# Observer 데이터 추출
broker_obs = None
for obs in strategy.observers:
    obs_name = type(obs).__name__
    if 'Broker' in obs_name:
        broker_obs = obs
        break

# 날짜 데이터 추출
data_len = len(strategy.data)
dates = [strategy.data.datetime.date(-data_len + i + 1) for i in range(data_len)]

# 포트폴리오 가치 추출
values = []
if broker_obs is not None:
    value_array = broker_obs.lines.value.array
    values = list(value_array[:data_len])
else:
    values = [initial_cash] * data_len
    values[-1] = strategy.broker.getvalue()

# 지수화 (시작점 = 100)
btc_prices = df['close'].values
btc_indexed = (btc_prices / btc_prices[0]) * 100
portfolio_indexed = (np.array(values) / values[0]) * 100

# 드로다운 직접 계산
max_value = initial_cash
dd_values = []
for v in values:
    if v > max_value:
        max_value = v
    dd = (max_value - v) / max_value * 100 if max_value > 0 else 0
    dd_values.append(dd)

# 매수/매도 시점 추출 (4가지 액션 구분)
long_entry_dates, long_entry_prices = [], []  # 롱 진입
long_exit_dates, long_exit_prices = [], []  # 롱 청산
short_entry_dates, short_entry_prices = [], []  # 숏 진입
short_cover_dates, short_cover_prices = [], []  # 숏 커버

for trade in strategy.trade_history:
    trade_date = trade['date']
    trade_timestamp = pd.Timestamp(trade_date)

    if trade_timestamp in df.index:
        idx = df.index.get_loc(trade_timestamp)
        indexed_price = btc_indexed[idx]
    else:
        closest_idx = df.index.searchsorted(trade_timestamp)
        if closest_idx >= len(df):
            closest_idx = len(df) - 1
        indexed_price = btc_indexed[closest_idx]

    action = trade['action']
    if action == '롱 진입':
        long_entry_dates.append(trade_date)
        long_entry_prices.append(indexed_price)
    elif action == '롱 청산':
        long_exit_dates.append(trade_date)
        long_exit_prices.append(indexed_price)
    elif action == '숏 진입':
        short_entry_dates.append(trade_date)
        short_entry_prices.append(indexed_price)
    elif action == '숏 커버':
        short_cover_dates.append(trade_date)
        short_cover_prices.append(indexed_price)

# 그래프 그리기 (2개 subplot)
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# 1. BTC 가격 vs 포트폴리오 가치 (지수화)
ax1 = axes[0]
ax1.plot(df.index, btc_indexed, label='BTC (Buy & Hold)', color='blue', alpha=0.7)
ax1.plot(dates, portfolio_indexed, label='EMA 정배열 전략', color='green', linewidth=2)
ax1.axhline(y=100, color='gray', linestyle='--', alpha=0.5, label='시작점 (100)')

# 롱/숏 마커 표시 (SHOW_MARKERS가 True일 때만)
if SHOW_MARKERS:
    if long_entry_dates:
        ax1.scatter(long_entry_dates, long_entry_prices, marker='^', color='#00AA00', s=150,
                    label=f'[L] 롱 진입 ({len(long_entry_dates)}회)', zorder=5, edgecolors='black', linewidths=1.5)
    if long_exit_dates:
        ax1.scatter(long_exit_dates, long_exit_prices, marker='v', facecolors='none', s=150,
                    label=f'[L] 롱 청산 ({len(long_exit_dates)}회)', zorder=5, edgecolors='#00AA00', linewidths=2.5)
    if short_entry_dates:
        ax1.scatter(short_entry_dates, short_entry_prices, marker='v', color='#DD0000', s=150,
                    label=f'[S] 숏 진입 ({len(short_entry_dates)}회)', zorder=5, edgecolors='black', linewidths=1.5)
    if short_cover_dates:
        ax1.scatter(short_cover_dates, short_cover_prices, marker='^', facecolors='none', s=150,
                    label=f'[S] 숏 커버 ({len(short_cover_dates)}회)', zorder=5, edgecolors='#DD0000', linewidths=2.5)

ax1.set_ylabel('지수 (시작 = 100)')
ax1.set_title('BTC Buy & Hold vs EMA 정배열 전략 성과 비교')
ax1.legend(loc='upper left', fontsize=9)
ax1.grid(True, alpha=0.3)

# 최종 수익률 표시
btc_final = btc_indexed[-1]
portfolio_final = portfolio_indexed[-1]
ax1.annotate(f'BTC: {btc_final:.0f} ({(btc_final - 100):.0f}%)',
             xy=(df.index[-1], btc_final), fontsize=9, color='blue')
ax1.annotate(f'전략: {portfolio_final:.0f} ({(portfolio_final - 100):.0f}%)',
             xy=(df.index[-1], portfolio_final), fontsize=9, color='green')

# 2. 드로다운
ax2 = axes[1]
ax2.fill_between(dates, dd_values, 0, color='red', alpha=0.3, label='드로다운')
ax2.set_ylabel('드로다운 (%)')
ax2.set_xlabel('날짜')
ax2.legend(loc='lower left')
ax2.grid(True, alpha=0.3)
ax2.invert_yaxis()

# X축 날짜 포맷 (연도 + 월)
ax2.xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # 3개월 간격
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# 손실 패턴 분석
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

strategy = result[0]

# 개별 거래 분석을 위한 데이터 수집
trades_analyzer = strategy.analyzers.getbyname('trades').get_analysis()

print("\n" + "=" * 60)
print("손실 패턴 상세 분석")
print("=" * 60)

# trade_history에서 거래 쌍 매칭 (진입 → 청산)
trade_pairs = []
open_trade = None

for trade in strategy.trade_history:
    action = trade['action']
    
    if action in ['롱 진입', '숏 진입']:
        open_trade = {
            'entry_date': trade['date'],
            'entry_price': trade['price'],
            'direction': '롱' if action == '롱 진입' else '숏'
        }
    elif action in ['롱 청산', '숏 커버'] and open_trade:
        open_trade['exit_date'] = trade['date']
        open_trade['exit_price'] = trade['price']
        
        # 수익률 계산
        if open_trade['direction'] == '롱':
            pnl_pct = (trade['price'] - open_trade['entry_price']) / open_trade['entry_price'] * 100
        else:  # 숏
            pnl_pct = (open_trade['entry_price'] - trade['price']) / open_trade['entry_price'] * 100
        
        # 보유 기간 계산 (일)
        hold_days = (open_trade['exit_date'] - open_trade['entry_date']).days
        
        open_trade['pnl_pct'] = pnl_pct
        open_trade['hold_days'] = hold_days
        open_trade['is_loss'] = pnl_pct < 0
        
        trade_pairs.append(open_trade)
        open_trade = None

# DataFrame 생성
if trade_pairs:
    trades_df = pd.DataFrame(trade_pairs)
    
    # 1. 기본 통계
    total_trades = len(trades_df)
    loss_trades = trades_df[trades_df['is_loss']]
    win_trades = trades_df[~trades_df['is_loss']]
    
    print(f"\n[1. 기본 통계]")
    print(f"   총 거래 수: {total_trades}")
    print(f"   손실 거래: {len(loss_trades)} ({len(loss_trades)/total_trades*100:.1f}%)")
    print(f"   수익 거래: {len(win_trades)} ({len(win_trades)/total_trades*100:.1f}%)")
    
    # 2. 보유 기간 분석
    print(f"\n[2. 보유 기간 분석]")
    print(f"   전체 평균 보유 기간: {trades_df['hold_days'].mean():.1f}일")
    print(f"   손실 거래 평균 보유: {loss_trades['hold_days'].mean():.1f}일")
    print(f"   수익 거래 평균 보유: {win_trades['hold_days'].mean():.1f}일")
    
    # 3. 휩소 패턴 분석 (짧은 보유 + 손실)
    short_hold_threshold = 5  # 5일 이하를 '짧은 보유'로 정의
    whipsaw_trades = loss_trades[loss_trades['hold_days'] <= short_hold_threshold]
    
    print(f"\n[3. 휩소 패턴 분석 (보유 {short_hold_threshold}일 이하 + 손실)]")
    print(f"   휩소 거래 수: {len(whipsaw_trades)}")
    print(f"   휩소 비율 (전체 대비): {len(whipsaw_trades)/total_trades*100:.1f}%")
    print(f"   휩소 비율 (손실 대비): {len(whipsaw_trades)/len(loss_trades)*100:.1f}%" if len(loss_trades) > 0 else "")
    if len(whipsaw_trades) > 0:
        print(f"   휩소 평균 손실률: {whipsaw_trades['pnl_pct'].mean():.2f}%")
    
    # 4. 롱/숏 별 성과
    long_trades = trades_df[trades_df['direction'] == '롱']
    short_trades = trades_df[trades_df['direction'] == '숏']
    
    print(f"\n[4. 롱/숏 별 성과]")
    if len(long_trades) > 0:
        long_win_rate = len(long_trades[~long_trades['is_loss']]) / len(long_trades) * 100
        print(f"   롱 거래: {len(long_trades)}회, 승률 {long_win_rate:.1f}%, 평균 {long_trades['pnl_pct'].mean():.2f}%")
    if len(short_trades) > 0:
        short_win_rate = len(short_trades[~short_trades['is_loss']]) / len(short_trades) * 100
        print(f"   숏 거래: {len(short_trades)}회, 승률 {short_win_rate:.1f}%, 평균 {short_trades['pnl_pct'].mean():.2f}%")
    
    # 5. 손실 규모 분석
    print(f"\n[5. 손실 규모 분석]")
    if len(loss_trades) > 0:
        loss_pcts = loss_trades['pnl_pct']
        print(f"   평균 손실률: {loss_pcts.mean():.2f}%")
        print(f"   최대 손실률: {loss_pcts.min():.2f}%")
        print(f"   손실 중앙값: {loss_pcts.median():.2f}%")
        
        # 큰 손실 (10% 이상)
        big_losses = loss_trades[loss_trades['pnl_pct'] <= -10]
        print(f"   큰 손실 (-10% 이상): {len(big_losses)}회")
    
    # 6. 연속 손실 분석
    trades_df['consecutive_loss'] = 0
    consecutive = 0
    max_consecutive = 0
    for i, row in trades_df.iterrows():
        if row['is_loss']:
            consecutive += 1
            max_consecutive = max(max_consecutive, consecutive)
        else:
            consecutive = 0
        trades_df.at[i, 'consecutive_loss'] = consecutive
    
    print(f"\n[6. 연속 손실 분석]")
    print(f"   최대 연속 손실: {max_consecutive}회")
    
    # 시각화
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. 거래별 손익률 분포
    ax1 = axes[0, 0]
    colors = ['green' if x >= 0 else 'red' for x in trades_df['pnl_pct']]
    ax1.bar(range(len(trades_df)), trades_df['pnl_pct'], color=colors, alpha=0.7)
    ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax1.set_xlabel('거래 번호')
    ax1.set_ylabel('수익률 (%)')
    ax1.set_title('거래별 손익률')
    ax1.grid(True, alpha=0.3)
    
    # 2. 보유 기간 vs 손익률 산점도
    ax2 = axes[0, 1]
    colors_scatter = ['green' if x >= 0 else 'red' for x in trades_df['pnl_pct']]
    ax2.scatter(trades_df['hold_days'], trades_df['pnl_pct'], c=colors_scatter, alpha=0.6, s=50)
    ax2.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
    ax2.axvline(x=short_hold_threshold, color='orange', linestyle='--', label=f'휩소 기준 ({short_hold_threshold}일)')
    ax2.set_xlabel('보유 기간 (일)')
    ax2.set_ylabel('수익률 (%)')
    ax2.set_title('보유 기간 vs 손익률 (빨간점=손실)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. 손익률 히스토그램
    ax3 = axes[1, 0]
    ax3.hist(trades_df['pnl_pct'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
    ax3.axvline(x=0, color='red', linestyle='--', linewidth=2, label='손익분기')
    ax3.axvline(x=trades_df['pnl_pct'].mean(), color='green', linestyle='--', linewidth=2, label=f"평균 ({trades_df['pnl_pct'].mean():.1f}%)")
    ax3.set_xlabel('수익률 (%)')
    ax3.set_ylabel('빈도')
    ax3.set_title('손익률 분포')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. 롱/숏 비교 박스플롯
    ax4 = axes[1, 1]
    box_data = []
    box_labels = []
    if len(long_trades) > 0:
        box_data.append(long_trades['pnl_pct'])
        box_labels.append(f'롱 (n={len(long_trades)})')
    if len(short_trades) > 0:
        box_data.append(short_trades['pnl_pct'])
        box_labels.append(f'숏 (n={len(short_trades)})')
    
    if box_data:
        bp = ax4.boxplot(box_data, tick_labels=box_labels, patch_artist=True)
        colors_box = ['lightgreen', 'lightcoral']
        for patch, color in zip(bp['boxes'], colors_box[:len(box_data)]):
            patch.set_facecolor(color)
    ax4.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
    ax4.set_ylabel('수익률 (%)')
    ax4.set_title('롱/숏 손익률 비교')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 7. 손실 거래 상세 목록 (상위 10개)
    print(f"\n[7. 최대 손실 거래 TOP 10]")
    top_losses = trades_df.nsmallest(10, 'pnl_pct')[['entry_date', 'exit_date', 'direction', 'hold_days', 'pnl_pct']]
    print(top_losses.to_string())
    
else:
    print("거래 기록이 없습니다.")

print("\n" + "=" * 60)